# 4.9: Intro to Data Visualization with Python

### This script contains the following points:

##### 01. Importing
##### 02. Data Wrangling
##### 03. Data Quality and Consistency Checks
##### 04. Merging Data
##### 05. Exporting

## 01. Importing

In [2]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
# folder path to main project folder
path = r'C:\Users\ThinkPad T570\Documents\01-2024 Instacart Basket Analysis'

In [4]:
# importing customers.cvs dataframe
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [14]:
df_customers.head()

,user_id,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [9]:
df_customers.shape

(206209, 10)

## 02. Data Wrangling

In [11]:
# renaming columns
df_customers.rename(columns={'Surnam': 'Surname', 'STATE': 'State'}, inplace=True)

In [12]:
# displaying column names
df_customers.columns

Index(['user_id', 'First Name', 'Surname', 'Gender', 'State', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [30]:
# exploring distribution in specific columns
df_customers['fam_status'].value_counts(dropna = False)

fam_status
married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: count, dtype: int64

## 03. Data Quality and Consistency Check

### Finding Missing Values

In [21]:
# checking for missing values in df_customers
df_customers.isnull().sum()

user_id             0
First Name      11259
Surname             0
Gender              0
State               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

In [22]:
# exploring missing observations
df_nan = df_customers[df_customers['First Name'].isnull()==True]

In [23]:
df_nan.head()

,user_id,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819


### Addressing Mixed-Type Data

In [28]:
# checking whether a dataframe contains any mixed-type columns
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].map(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print(col)

First Name


In [29]:
# defining "First Name" entries as string variable
df_customers['First Name'] = df_customers['First Name'].astype('str')

### Finding Duplicates

In [31]:
# Looking for full duplicates within a dataframe
df_dups = df_customers[df_customers.duplicated()]

In [32]:
df_dups

,user_id,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,income


no duplicates found

## 04. Merging Data

In [34]:
# checking the data typ of the "user_id" column
df_customers['user_id'].dtypes

dtype('int64')

In [35]:
# importing orders_products_flaged dataframe
df_ords_prods_flaged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_flaged.pkl'))

In [36]:
df_ords_prods_flaged.shape

(32404859, 24)

In [37]:
df_ords_prods_flaged.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,_merge,Busiest_days,busiest_day,busiest_period_of_day,max_order,loyalty_flag,avg.price_order,spending_flag,median_days_prior_order,ord_frequency_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,both,Regular days,Regularly busy,Most orders,32,Regular customer,6.935811,Low Spender,8.0,Frequent customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,both,Regular days,Regularly busy,Average orders,32,Regular customer,6.935811,Low Spender,8.0,Frequent customer
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,both,Busiest days,Busiest day,Average orders,5,New customer,7.930208,Low Spender,8.0,Frequent customer
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,...,both,Slowest days,Regularly busy,Most orders,3,New customer,4.972414,Low Spender,9.0,Frequent customer
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,both,Slowest days,Least busy,Average orders,3,New customer,4.972414,Low Spender,9.0,Frequent customer


In [38]:
# merging df_customers on df_ords_prods_flaged using the user_id as identifier
df_ords_prods_merge = df_ords_prods_flaged.merge(df_customers, on = 'user_id')

In [39]:
df_ords_prods_merge.shape

(32404859, 33)

In [42]:
# command to tell pandas not to assign any options regarding the maximum number of rows / columns to display
pd.options.display.max_columns = None

In [43]:
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,Busiest_days,busiest_day,busiest_period_of_day,max_order,loyalty_flag,avg.price_order,spending_flag,median_days_prior_order,ord_frequency_flag,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,11,3.0,5,0,both,Regular days,Regularly busy,Most orders,32,Regular customer,6.935811,Low Spender,8.0,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,17,20.0,1,1,both,Regular days,Regularly busy,Average orders,32,Regular customer,6.935811,Low Spender,8.0,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,prior,1,5,13,NaN,4,0,both,Regular days,Regularly busy,Most orders,32,Regular customer,6.935811,Low Spender,8.0,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,prior,10,5,14,6.0,1,1,both,Regular days,Regularly busy,Most orders,32,Regular customer,6.935811,Low Spender,8.0,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
4,1000,Apricots,18,10,12.9,505689,138,prior,9,6,12,7.0,5,0,both,Regular days,Regularly busy,Most orders,32,Regular customer,6.935811,Low Spender,8.0,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,1,married,49620


In [41]:
df_ords_prods_merge.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order', 'add_to_cart_order',
       'reordered', '_merge', 'Busiest_days', 'busiest_day',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'avg.price_order',
       'spending_flag', 'median_days_prior_order', 'ord_frequency_flag',
       'First Name', 'Surname', 'Gender', 'State', 'Age', 'date_joined',
       'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [44]:
df_ords_prods_merge_clear = df_ords_prods_merge.drop(['eval_set', '_merge', 'max_order', 'avg.price_order', 'median_days_prior_order'], axis=1)

In [46]:
df_ords_prods_merge_clear.shape

(32404859, 28)

In [45]:
df_ords_prods_merge_clear.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'add_to_cart_order', 'reordered',
       'Busiest_days', 'busiest_day', 'busiest_period_of_day', 'loyalty_flag',
       'spending_flag', 'ord_frequency_flag', 'First Name', 'Surname',
       'Gender', 'State', 'Age', 'date_joined', 'n_dependants', 'fam_status',
       'income'],
      dtype='object')

In [47]:
# rearranging the columns order
df_user_history = df_ords_prods_merge_clear[['user_id', 'First Name', 'Surname','Gender', 'Age', 'State', 'date_joined', 'fam_status', 'n_dependants', 'income', 'loyalty_flag', 'spending_flag', 'ord_frequency_flag', 'product_id', 'product_name', 'aisle_id', 'department_id', 'prices','order_id', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'add_to_cart_order', 'reordered', 'Busiest_days', 'busiest_day', 'busiest_period_of_day']]

In [48]:
df_user_history.head()

,user_id,First Name,Surname,Gender,Age,State,date_joined,fam_status,n_dependants,income,loyalty_flag,spending_flag,ord_frequency_flag,product_id,product_name,aisle_id,department_id,prices,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,Busiest_days,busiest_day,busiest_period_of_day
0,138,Charles,Cox,Male,81,Minnesota,8/1/2019,married,1,49620,Regular customer,Low Spender,Frequent customer,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,28,6,11,3.0,5,0,Regular days,Regularly busy,Most orders
1,138,Charles,Cox,Male,81,Minnesota,8/1/2019,married,1,49620,Regular customer,Low Spender,Frequent customer,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,30,6,17,20.0,1,1,Regular days,Regularly busy,Average orders
2,138,Charles,Cox,Male,81,Minnesota,8/1/2019,married,1,49620,Regular customer,Low Spender,Frequent customer,907,Premium Sliced Bacon,106,12,20.0,3160996,1,5,13,NaN,4,0,Regular days,Regularly busy,Most orders
3,138,Charles,Cox,Male,81,Minnesota,8/1/2019,married,1,49620,Regular customer,Low Spender,Frequent customer,907,Premium Sliced Bacon,106,12,20.0,2254091,10,5,14,6.0,1,1,Regular days,Regularly busy,Most orders
4,138,Charles,Cox,Male,81,Minnesota,8/1/2019,married,1,49620,Regular customer,Low Spender,Frequent customer,1000,Apricots,18,10,12.9,505689,9,6,12,7.0,5,0,Regular days,Regularly busy,Most orders


## 05. Exporting

In [49]:
# Exporting data to pkl
df_user_history.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'user_history.pkl'))